In [1]:
import torch
import numpy as np
import time

c:\Users\Sam\miniconda3\envs\wisp\lib\site-packages\tqdm-4.64.1-py3.9.egg\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def quantized_to_octree(quantized_points, level):

    if quantized_points.device!='cpu':
        quantized_points=quantized_points.detach().cpu()

    pts=np.array(torch.unique(quantized_points,dim=0), dtype=np.uint8)[:,None]
    bits = np.unpackbits(pts, 1)[...,[0,1,2]]
    bits = bits.reshape(len(bits),-1)

    m_idx=bits@np.power(2,np.arange(24)[ : :-1])

    oct=np.zeros((2**level)**3, dtype=bool)
    oct[m_idx]=1
    octree=[]

    for _ in range(level):
        octL = np.packbits(oct.reshape(-1,8),-1,bitorder='little')
        oct = octL>0
        octree.insert(0,octL[oct])

    return torch.tensor(np.concatenate(octree), dtype=torch.uint8, device='cuda')

In [42]:
levels=8
n=10000
occupied=torch.randint(0,2**levels,(n,3))

In [50]:
A=quantized_to_octree(occupied,levels)

In [44]:
occupied.device

device(type='cpu')

In [45]:
map=torch.pow(2,torch.arange(8).to(torch.uint8)[:,None])
map

tensor([[  1],
        [  2],
        [  4],
        [  8],
        [ 16],
        [ 32],
        [ 64],
        [128]], dtype=torch.uint8)

In [46]:
from kaolin import _C

def quantized_to_octree1(quantized_points, level):

    morton=_C.ops.spc.points_to_morton_cuda(quantized_points.contiguous().short().cuda())

    oct=torch.zeros((2**level)**3, dtype=bool)
    oct[morton]=1
    octree=[]

    for _ in range(level):
        octL = oct.reshape(-1,8).byte()@ map
        oct = octL>0
        octree.insert(0,octL[oct])

    return torch.tensor(np.concatenate(octree), dtype=torch.uint8, device='cuda')

In [51]:
B=quantized_to_octree1(occupied,levels)

In [52]:
torch.all(A==B)

tensor(True, device='cuda:0')

# CUDA KERNEL

In [6]:
import torch
import spc

In [7]:
occupied=torch.tensor(occupied).cuda().to(torch.int32)

C:\Users\Sam\AppData\Local\Temp\ipykernel_14084\1858587691.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  occupied=torch.tensor(occupied).cuda().to(torch.int32)


In [8]:
occupied.dtype

torch.int32

In [9]:
out= spc.spc(occupied,levels)

: 

: 

In [ ]:
out[1],out[2]

(tensor([      0,       1,       9,      73,     585,    4681,   37449,  299593,
         2396745], dtype=torch.int32),
 tensor([       1,        9,       73,      585,     4681,    37449,   299593,
          2396745, 19173961], dtype=torch.int32))

In [ ]:
out[0]

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8)

In [ ]:
out[3]

tensor([     100,        1,  2396745, 19173961,        0,        0,        0,
               0,        0], dtype=torch.int32)

In [ ]:
19173961-2396745

16777216

In [ ]:
out[4]